# Time Tracking

Our target user is a freelancer who bills clients by the time worked. Therefore, managing, tracking and planning your work time is a central task. Tuttle wants to make this as easy and effective as possible for you. Let's get started:

## Preamble

In [15]:
from pathlib import Path
import ipywidgets
from IPython import display
import datetime

In [16]:
import tuttle

In [17]:
controller = tuttle.controller.Controller(home_dir=".demo_home")

## How to Record Project Time with Tuttle

- Previously we asked you to assign a unique **tag** to each project that you want to track. We use this tag to assign time tracking entries to projects, trying to find the tag in:
    - the title of your calendar entries
    - the title of your time tracking entries

## Importing Time Tracking Data

Currently we provide several ways of inputting time management data:
    
1. **Cloud Calendar**: Querying your cloud calendar: Log into your cloud calendar provider and import your calendar events seamlessly.
2. **File Calendar**: Export your calendar in the iCalendar format and parse it.
3. **Time Tracking Spreadsheets**: You may prefer to track your time not in the calendar but a specialized tool. As long as this tool can export time tracking data in a consistent format, we want to provide an option to import it into Tuttle.

In [18]:
time_tracking_preference = ipywidgets.RadioButtons(options=["File Calendar", "Cloud Calendar", "Spreadsheet"])

_Select your prereference for importing time tracking data:_

In [19]:
time_tracking_preference

RadioButtons(options=('File Calendar', 'Cloud Calendar', 'Spreadsheet'), value='File Calendar')

### A) Time Tracking via Cloud Calendar

If you have an Apple iCloud account, follow this process. (We are working on connectors with other cloud services)

In [20]:
icloud_username = controller.user.icloud_account.user_name
icloud_username

'mail@tuttle.com'

Connect to your cloud calendar that contains your time tracking events:

In [21]:
from tuttle.calendar import ICloudCalendar

In [22]:
if time_tracking_preference.value == "Cloud Calendar":
    my_calendar = ICloudCalendar(
        icloud=tuttle.cloud.login_iCloud(
            user_name=icloud_username,
        ),
        name=input("calendar name: "),
    )

### B) Time  Tracking via Calendar Import

In [23]:
from tuttle.calendar import FileCalendar

**Example: file calendar**

In [24]:
timetracking_calendar_path = Path("../../tuttle_tests/data/TuttleDemo-TimeTracking.ics")

In [25]:
if time_tracking_preference.value == "File Calendar":
    my_calendar = FileCalendar(
        path=timetracking_calendar_path, 
        name="TimeTracking"
    )

### C) Time Tracking via Data Export from Time Tracking Tools

In [26]:
if time_tracking_preference.value == "Spreadsheet":
    timetracking_data = tuttle.timetracking.import_from_spreadsheet(
        path="../../tuttle_tests/data/test_time_tracking_toggl.csv",
        preset=tuttle.timetracking.TimetrackingSpreadsheetPreset.Toggl,
    )

Check if the format of your spreadsheets is already supoorted as a preset. Otherwise, set the arguments of `import_from_spreadsheet` to match your required format.

After a successful import, you can pass the resulting `timetracking_data` wherever as a `source` to the following functions.

## Generating Time Sheets

Now that we have connected a source for time tracking data, we can pass it to other functions. In the following we generate **time sheets**. 

A **time sheet** 
- can be rendered to a document for reporting purposes
- is the basis for an **invoice** - more on invoicing later.

**Select a project**

**Example**

In [27]:
my_project = controller.get_project(title="Heating Repair")

_Your turn: Select one of your projects by title_

**Generate a time sheet**

In [28]:
my_timesheet = tuttle.timetracking.generate_timesheet(
    source=my_calendar,
    project=my_project,
    period_start="February 2022",
    period_end="March 2022",
    item_description=my_project.title,
)

/Users/cls/Documents/Work/Projects/PrototypeFund/Dev/tuttle/tuttle/timetracking.py:56: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  timetracking_data.loc[period_start:period_end].query(tag_query).sort_index()
/Users/cls/miniforge3/envs/tuttle-flet/lib/python3.10/site-packages/sqlmodel/orm/session.py:101: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return super().execute(  # type: ignore


_Your turn: Generate a timesheet for your project_

**Display the timesheet data**

In [29]:
my_timesheet.table

,id,timesheet_id,begin,end,duration,title,tag,description
0,None,None,2022-02-17 01:00:00+01:00,2022-02-18 01:00:00+01:00,0 days 08:00:00,#HeatingRepair,#HeatingRepair,Heating Repair
1,None,None,2022-02-18 01:00:00+01:00,2022-02-19 01:00:00+01:00,0 days 08:00:00,#HeatingRepair,#HeatingRepair,Heating Repair


**Render the timesheet document**

The following function renders a timesheet to an HTML-based layout:

You can display the HTML in the notebook...

In [30]:
display.HTML(
    tuttle.rendering.render_timesheet(
        user=controller.user,
        timesheet=my_timesheet,
    )
)

IntegrityError: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) NOT NULL constraint failed: timesheet.period
[SQL: INSERT INTO timesheet (title, date, period, project_id, comment) VALUES (?, ?, ?, ?, ?)]
[parameters: ('Heating Repair - February 2022 - March 2022', '2022-09-07', None, 1, '')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

... or render it to a file:

_Set the path to a folder where you want your invoices to appear_:

In [ ]:
timesheet_dir = Path.home() / "Downloads"

In [ ]:
tuttle.rendering.render_timesheet(
    user=controller.user,
    timesheet=my_timesheet,
    style="anvil",
    out_dir=timesheet_dir,
)

This will create a folder named with the timesheet title, containing the timesheet as an HTML document.

In [ ]:
timesheet_path = str(timesheet_dir / f"Invoice-{my_timesheet.title}" / f"Timesheet-{my_timesheet.title}.html")

You can also render the timesheet to PDF. For now, this requires the native [wkhtmltopdf](https://wkhtmltopdf.org) tool to be installed.

In [ ]:
tuttle.rendering.render_timesheet(
    user=controller.user,
    timesheet=my_timesheet,
    style="anvil",
    out_dir=timesheet_dir,
    document_format="pdf",
)

## Invoicing

Now that we have set up our user info, clients, contracts and projects, as well as a source for time tracking data, we are ready to automatically generate invoices.

### Workflow

_1. Select a project_

**Example**

In [ ]:
my_project = controller.get_project(title="Heating Repair")

2. Generate an invoice for one or more timesheet(s).

**Example**

In [ ]:
my_invoice = tuttle.invoicing.generate_invoice(
    timesheets=[
        my_timesheet,
    ],
    contract=my_project.contract,
    date=datetime.date.today(),
)

In [ ]:
my_invoice.number

In [ ]:
my_invoice.total

5. Render the invoice to a document template:

You can display the HTML in the notebook...

In [ ]:
display.HTML(
    tuttle.rendering.render_invoice(
        user=controller.user, 
        invoice=my_invoice,
        style=None
    )
)

... or render it to a file:

_Set the path to a folder where you want your invoices to appear_:

In [ ]:
invoice_dir = Path.home() / "Downloads"

In [ ]:
tuttle.rendering.render_invoice(
    user=controller.user, 
    invoice=my_invoice,
    style="anvil",
    out_dir=invoice_dir,
)

This will create a folder named with the inovice number, containing the invoice as an HTML document.

In [ ]:
invoice_path = str(invoice_dir / f"Invoice-{my_invoice.number}" / f"Invoice-{my_invoice.number}.html")

_Your turn:_

You can also render the invoice to PDF. For now, this requires the native [wkhtmltopdf](https://wkhtmltopdf.org) tool to be installed.

In [ ]:
tuttle.rendering.render_invoice(
    user=controller.user, 
    invoice=my_invoice,
    style="anvil",
    out_dir=invoice_dir,
    document_format="pdf"
)

### Send the Invoice Automatically

![](img/underconstruction.jpg)

------